 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [12]:
# Reading whale returns
whale_returns_data = Path(r"C:\Users\tfree\Desktop\MIA Fintech bc\MIA-VIRT-FIN-PT-08-2023-U-LOLC\04-Pandas\Homework Instructions\Starter_Code\Resources\whale_returns.csv")
whale_returns = pd.read_csv(whale_returns_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
whale_returns.sort_index(inplace=True)
whale_returns.head()

C:\Users\tfree\AppData\Local\Temp\ipykernel_7576\182841394.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  whale_returns = pd.read_csv(whale_returns_data, index_col="Date", infer_datetime_format=True, parse_dates=True)


,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC
Date,,,,
2015-03-02,NaN,NaN,NaN,NaN
2015-03-03,-0.001266,-0.004981,-0.000496,-0.006569
2015-03-04,0.002230,0.003241,-0.002534,0.004213
2015-03-05,0.004016,0.004076,0.002355,0.006726
2015-03-06,-0.007905,-0.003574,-0.008481,-0.013098


In [ ]:
# Count nulls
whale_returns.isnull().sum().sum()

In [ ]:
# Drop nulls
whale_returns = whale_returns.dropna()


## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_data = Path(r"C:\Users\tfree\Desktop\MIA Fintech bc\MIA-VIRT-FIN-PT-08-2023-U-LOLC\04-Pandas\Homework Instructions\Starter_Code\Resources\algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
algo_returns.sort_index(inplace=True)
algo_returns.head()

In [ ]:
# Count nulls
algo_returns.isnull().sum().sum()

In [ ]:
# Drop nulls
algo_returns = algo_returns.dropna()
algo_returns.isnull().sum()


## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_data = Path(r"C:\Users\tfree\Desktop\MIA Fintech bc\MIA-VIRT-FIN-PT-08-2023-U-LOLC\04-Pandas\Homework Instructions\Starter_Code\Resources\sp500_history.csv")
sp500_history = pd.read_csv(sp500_history_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_history.sort_index(ascending =True, inplace=True)
sp500_history.head()

sp500_history["Close"] = sp500_history["Close"].str.replace("$", "")
sp500_history["Close"]

In [ ]:
# Check Data Types
sp500_history.dtypes

In [ ]:
# Fix Data Types
sp500_history["Close"] = sp500_history["Close"].astype("float")

In [ ]:
# Calculate Daily Returns
sp500_returns = (sp500_history - sp500_history.shift(1)) / sp500_history.shift(1)
print(sp500_returns.head())

In [ ]:
# Drop nulls
sp500_returns.isnull().sum().sum()
sp500_returns = sp500_returns.dropna()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp500_returns.rename(columns={'Close':'SP500 Return'}, inplace=True)
print(sp500_returns.head())


## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
whale_returns = whale_returns[whale_returns.index.isin(sp500_history.index)]
algo_returns = algo_returns[algo_returns.index.isin(sp500_history.index)]

combined_returns = pd.concat([whale_returns, algo_returns, sp500_returns], axis=1)

combined_returns.describe()


---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_returns.plot(figsize=(10,5))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + combined_returns).cumprod()
cumulative_returns.head()

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
cumulative_returns.plot.box()

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
portfolio_stddev = combined_returns.std()

print("Standard Deviations of Portfolios:")
print(portfolio_stddev)


### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500
sp500_std = sp500_returns.std()
print(sp500_std)
# Determine which portfolios are riskier than the S&P 500

sp500_stddev = combined_returns['Daily Return'].std()
portfolio_stddev_df = pd.DataFrame(portfolio_stddev, columns=['Standard Deviation'])
riskier_portfolios = portfolio_stddev_df[portfolio_stddev_df['Standard Deviation'] > sp500_stddev]
print("Portfolios Riskier Than S&P 500:")
print(riskier_portfolios)

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
sp500_daily_stddev = combined_returns['Daily Return'].std()
trading_days_per_year = 252
sp500_annual_stddev = sp500_daily_stddev * (trading_days_per_year ** 0.5)
print("Annualized Standard Deviation of S&P 500:")
print(sp500_annual_stddev)


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_std = combined_returns.rolling(window=21).std()

# Plot the rolling standard deviation
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
rolling_std.plot(title="Rolling 21-Day Standard Deviation of Portfolios", figsize=(12, 6))
plt.xlabel("Date")
plt.ylabel("Standard Deviation")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation_matrix = combined_returns.corr()

# Display de correlation matrix

import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Heatmap of Portfolios")
plt.tight_layout()
plt.show()


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
portfolio_returns = combined_returns['BERKSHIRE HATHAWAY INC']
portfolio_covariance = portfolio_returns.cov(combined_returns['Daily Return'])

# Calculate variance of S&P 500
sp500_variance = combined_returns['Daily Return'].var()
print(sp500_variance)

# Computing beta
beta = portfolio_covariance / sp500_variance
print("Beta is:", beta)

# Plot beta trend

rolling_window = 60
rolling_beta = portfolio_returns.rolling(window=rolling_window).cov(combined_returns['Daily Return']) / sp500_variance

import seaborn as sns

rolling_window = 60
rolling_beta = portfolio_returns.rolling(window=rolling_window).cov(combined_returns['Daily Return']) / sp500_variance

plt.figure(figsize=(12, 6))
sns.lineplot(data=rolling_beta, color='blue', label=f'Rolling {rolling_window}-Day Beta')
plt.axhline(beta, color='red', linestyle='--', label='Current Beta')
plt.title(f'Rolling {rolling_window}-Day Beta of {portfolio_name} vs. S&P 500')
plt.xlabel('Date')
plt.ylabel('Beta')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window

ewma_half_life = 21
ewma_std = combined_returns.rolling(window=ewma_half_life).std()

plt.figure(figsize=(12, 6))
for portfolio in combined_returns.columns:
    plt.plot(ewma_std[portfolio], label=portfolio)

plt.title(f'EWMA of Standard Deviation ({ewma_half_life}-Day Half-Life)')
plt.xlabel('Date')
plt.ylabel('EWMA of Standard Deviation')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
aapl_historical_path = Path(r"../Starter_Code/Resources/aapl_historical.csv")
aapl_df = pd.read_csv(aapl_historical_path)
aapl_df.head(10)

In [ ]:
# Reading data from 2nd stock
goog_historical_path = Path(r"../Starter_Code/Resources/goog_historical.csv")
goog_df = pd.read_csv(goog_historical_path)
goog_df.head(10)

In [ ]:
# Reading data from 3rd stock
cost_historical_path = Path(r"../Starter_Code/Resources/cost_historical.csv")
cost_df = pd.read_csv(cost_historical_path)
cost_df.head(10)

In [ ]:
# Combine all stocks in a single DataFrame
combined_df = pd.concat([goog_df, aapl_df, cost_df], axis="columns", join="inner", ignore_index=False)
combined_df.sort_index(inplace=True)
combined_df.head(10)

In [ ]:
# Reset Date index
combined_df.set_index(pd.to_datetime(combined_df['Trade DATE'], infer_datetime_format=True), inplace=True)
combined_df.head()

In [ ]:
# Reorganize portfolio data by having a column per symbol
df_pivot = combined_df.pivot_table(index= "index", columns="Symbol", values="NOCP", aggfunc='first')
df_pivot.reset_index(inplace=True)
df_pivot.columns.name = None

print(df_pivot)

In [ ]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!